# Neural network for dose measurement

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split
import numpy as np
import pandas as pd
%matplotlib tk

import matplotlib.pyplot as plt
import NNPytorchLightning as NNPL

Here loading trained models to compare the training and validation losses per model, and look at performance on unseen data. These models are as follows:

model_cubes: trained just on cube counts, with a batch size of 200 and 200 samples per data set, learning rate of 1e-3, for 500 epochs (for time)

model_cubes_profiles: trained on cube counts and profiles, with a batch size of 200 and 200 samples per data set, learning rate of 1e-3, for 500 epochs (for time)

In [2]:
coeffs = '/home/nr1315/Documents/Project/effective_dose_coeffs.h5'
energy_bins = '/home/nr1315/Documents/Project/MachineLearning/energy_bins.npy'

model_cubes = NNPL.LoadModel('/home/nr1315/Documents/Project/MachineLearning/lightning_logs/model_cubes_new_data/version_1/',torch.rand((1,1,64)),coeffs,energy_bins)

model_cubes_profiles = NNPL.LoadModel('/home/nr1315/Documents/Project/MachineLearning/lightning_logs/model_cubes_profiles_new_data/version_5/',torch.rand((1,1,76)),coeffs,energy_bins)

Also need to load the loss curves separately, due to the way they are from the logs.

In [3]:
model_cubes_tloss = pd.read_csv('/home/nr1315/Downloads/run-model_cubes_new_data_version_1-tag-train_loss.csv')
model_cubes_vloss = pd.read_csv('/home/nr1315/Downloads/run-model_cubes_new_data_version_1-tag-val_loss.csv')

model_cubes_profiles_tloss = pd.read_csv('/home/nr1315/Downloads/run-model_cubes_profiles_new_data_version_5-tag-train_loss.csv')
model_cubes_profiles_vloss = pd.read_csv('/home/nr1315/Downloads/run-model_cubes_profiles_new_data_version_5-tag-val_loss.csv')

We also define the directories from which to load the testing data, for convenience:

In [4]:
testing_data_dir = '/home/nr1315/Documents/Project/MachineLearning/TestingData/'

AmBe_counts = 'SimCubeCounts_AmBe_5_0-0-0-0-1-0_1500.npy'
AmLi_counts = 'SimCubeCounts_AmLi_5_0-0-0-0-1-0_1500.npy'
Cf_counts = 'SimCubeCounts_Cf252_6_0-0-0-0-1-0_1500.npy'

AmBe_target = 'SimEnergyBins_AmBe.npy'
AmLi_target = 'SimEnergyBins_AmLi.npy'
Cf_target = 'SimEnergyBins_Cf252.npy'

For each model in turn, we will look at the loss curves and prediction on AmBe, AmLi, and Cf-252. 

### Cubes only model, 500 epochs, 200 samples per dataset



In [12]:
NNPL.PlotLosses([model_cubes_tloss,model_cubes_vloss],['Training loss','Validation loss'],'Model with only cube counts')

![Loss curves](Plots/model_cubes_loss.png)

In [6]:
fig,ax = plt.subplots(1,3,figsize=(30,10))

NNPL.compare_pred_true(model_cubes,testing_data_dir+AmBe_counts,testing_data_dir+AmBe_target,'AmBe','cubes only model',ax[0],24,np.load(energy_bins))
NNPL.compare_pred_true(model_cubes,testing_data_dir+AmLi_counts,testing_data_dir+AmLi_target,'AmLi','cubes only model',ax[1],24,np.load(energy_bins))
NNPL.compare_pred_true(model_cubes,testing_data_dir+Cf_counts,testing_data_dir+Cf_target,r'$^{252}$Cf','cubes only model',ax[2],24,np.load(energy_bins))

![Predictions of test data](Plots/simulated_predictions_cubes_model.png)

### Cubes and profiles model, 200 samples per dataset, learning rate 0.001

In [7]:
NNPL.PlotLosses([model_cubes_profiles_1_tloss,model_cubes_profiles_1_vloss],['Training loss','Validation loss'],'Cubes and profiles model, lr = 0.001, 200 samples per dataset')

![losses](Plots/model_cubes_profiles_200samples.png)

In [8]:
fig,ax = plt.subplots(1,3,figsize=(30,10))

NNPL.compare_pred_true(model_cubes_profiles_1,testing_data_dir+'withProfiles/'+AmBe_counts,testing_data_dir+AmBe_target,'AmBe','cubes only model',ax[0],24,np.load(energy_bins))
NNPL.compare_pred_true(model_cubes_profiles_1,testing_data_dir+'withProfiles/'+AmLi_counts,testing_data_dir+AmLi_target,'AmLi','cubes only model',ax[1],24,np.load(energy_bins))
NNPL.compare_pred_true(model_cubes_profiles_1,testing_data_dir+'withProfiles/'+Cf_counts,testing_data_dir+Cf_target,r'$^{252}$Cf','cubes only model',ax[2],24,np.load(energy_bins))

![cubes profiles 200 samples prediction](Plots/model_cubes_profiles_200samples_prediction.png)